In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

# Current directory
print(os.getcwd())

# change directory
os.chdir('/content/drive/MyDrive/python_training/NLP100Days/project_2_news_classf_2')
print(os.getcwd())

/content
/content/drive/MyDrive/python_training/NLP100Days/project_2_news_classf_2


In [3]:
!python --version

Python 3.7.10


In [4]:
!pip install -q -U pip
!pip install -q numpy
!pip install -q pandas
!pip install -q ckiptagger
!pip install -q tqdm
!pip install -q tensorflow==1.14.0
!pip install -q ipywidgets
!pip install -q matplotlib

     |████████████████████████████████| 1.5MB 5.9MB/s 
     |████████████████████████████████| 109.3 MB 46 kB/s 
     |████████████████████████████████| 50 kB 5.9 MB/s 
     |████████████████████████████████| 488 kB 70.1 MB/s 
     |████████████████████████████████| 3.1 MB 51.5 MB/s 


In [5]:
import pandas as pd
import numpy as np

from ckiptagger import WS, POS
from tqdm.notebook import tqdm

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [6]:
!ls

news_clustering_test.tsv   project_2_分類器_進階篇_hw.ipynb
news_clustering_train.tsv


In [7]:
df_train = pd.read_csv('news_clustering_train.tsv', sep='\t')
df_test = pd.read_csv('news_clustering_test.tsv', sep='\t')

In [8]:
df_train[:5]

,index,class,title
0,0,體育,亞洲杯奪冠賠率：日本、伊朗領銜 中國竟與泰國並列
1,1,體育,9輪4球本土射手僅次武磊 黃紫昌要搶最強U23頭銜
2,2,體育,如果今年勇士奪冠，下賽季詹姆斯何去何從？
3,3,體育,超級替補！科斯塔本賽季替補出場貢獻7次助攻
4,4,體育,騎士6天里發生了啥？從首輪搶七到次輪3-0猛龍


In [9]:
df_test[:5]

,index,class,title
0,1800,體育,如果騎士火箭進入總決賽，誰的勝算大？
1,1801,體育,從個人競技狀態來看，三個階段的詹姆斯，哪個最強？
2,1802,體育,騎士總冠軍！地球人誰能阻擋詹姆斯？史上最佳就是他！打服所有人
3,1803,體育,詹姆斯絕殺，騎士3比0，猛龍懷疑人生
4,1804,體育,騎士和步行者戰成搶七險勝，而猛龍即將被橫掃，步行者跟猛龍的區別在哪裡？


In [10]:
train_titles = {row['index']: row['title'] for _, row in df_train.iterrows()}
train_classes = {row['index']: row['class'] for _, row in df_train.iterrows()}

test_titles = {row['index']: row['title'] for _, row in df_test.iterrows()}
test_classes = {row['index']: row['class'] for _, row in df_test.iterrows()}

train_titles:

{0: '亞洲杯奪冠賠率：日本、伊朗領銜 中國竟與泰國並列',

 1: '9輪4球本土射手僅次武磊 黃紫昌要搶最強U23頭銜',

 2: '如果今年勇士奪冠，下賽季詹姆斯何去何從？',

 3: '超級替補！科斯塔本賽季替補出場貢獻7次助攻' ,

 ...
}


train_classes:

{0: '體育',

 1: '體育',

 2: '體育',

 3: '體育',

 4: '體育',

 5: '體育',

 ...
}

In [16]:
all_news_class = ['體育', '財經', '科技', '旅遊', '農業', '遊戲']

# 斷詞 + POS

In [18]:
ws = WS('../ckip_data/')
pos = POS('../ckip_data/')

In [19]:
train_title_cuts = {}
for index, title in tqdm(train_titles.items()):
    word_s = ws([title])
    word_p = pos(word_s)
    train_title_cuts[index] = list(zip(word_s[0], word_p[0]))

In [20]:
test_title_cuts = {}
for index, title in tqdm(test_titles.items()):
    word_s = ws([title])
    word_p = pos(word_s)
    test_title_cuts[index] = list(zip(word_s[0], word_p[0]))

train_title_cuts:

{0: [('亞洲杯', 'Nb'),

  ('奪', 'VC'),

  ('冠賠率', 'Na'),

  ('：', 'COLONCATEGORY'),

  ('日本', 'Nc'),

  ('、', 'PAUSECATEGORY'),

  ('伊朗', 'Nc'),

  ('領銜', 'VB'),

  (' ', 'WHITESPACE'),

  ('中國', 'Nc'),

  ('竟', 'D'),

  ('與', 'P'),

  ('泰國', 'Nc'),

  ('並列', 'VG')],

 1: [('9', 'Neu'),

# Bag of Words (BOW)

In [22]:
word2index = {}
index2word = {}
n = 0
for index in train_title_cuts:
    for word, flag in train_title_cuts[index]:
        if word in word2index:
            continue
        word2index[word] = n 
        index2word[n] = word
        n += 1

In [23]:
def get_bow_vector_with_selection(pairs, word2index):
    excluded_flags = [
        'Nh', 'Nep', 'Nes', 'DE', 'T', 'P', 'V_2', 'SHI',
        'Dfa', 'Dfb', 'Da', 'Di', 'Dk',
        'Caa', 'Cab', 'Cba', 'Cbb',
        'COLONCATEGORY', 'COMMACATEGORY', 'DASHCATEGORY', 'DOTCATEGORY', 'ETCCATEGORY', 'EXCLAMATIONCATEGORY',
        'PARENTHESISCATEGORY', 'PAUSECATEGORY', 'PERIODCATEGORY', 'QUESTIONCATEGORY', 'SEMICOLONCATEGORY',
        'SPCHANGECATEGORY', 'WHITESPACE'
    ]
    vector = np.zeros(len(word2index))
    for word, flag in pairs:
        if word in word2index and flag not in excluded_flags:
            vector[word2index[word]] += 1
    return vector

In [24]:
train_bow_vectors = {
    index: get_bow_vector_with_selection(pairs, word2index)
    for index, pairs in train_title_cuts.items()
}

test_bow_vectors = {
    index: get_bow_vector_with_selection(pairs, word2index)
    for index, pairs in test_title_cuts.items()
}

train_bow_vectors:

{0: array([1., 1., 1., ..., 0., 0., 0.]),

 1: array([0., 0., 0., ..., 0., 0., 0.]),
 

# TFIDF

train_vector_array:

(array([1., 1., 1., ..., 0., 0., 0.]),

 array([0., 0., 0., ..., 0., 0., 0.]),

 array([0., 0., 0., ..., 0., 0., 0.]),

 array([0., 0., 0., ..., 0., 0., 0.]),

 array([0., 0., 0., ..., 0., 0., 0.]),
 .....)

In [26]:
from sklearn.feature_extraction.text import TfidfTransformer

In [29]:
train_index_array, train_vector_array = list(zip(*train_bow_vectors.items()))

# 使用`TfidfTransformer`來將`train_vector_array`轉換成`train_tfidf_vector_array`
# YOUR CODE HERE

transformer = TfidfTransformer()
train_tfidf_vector_array = transformer.fit_transform(train_vector_array)

# END YOUR CODE

In [31]:
train_tfidf_vector_array

<1800x6690 sparse matrix of type '<class 'numpy.float64'>'
	with 17126 stored elements in Compressed Sparse Row format>

In [32]:
type(train_tfidf_vector_array)

scipy.sparse.csr.csr_matrix

In [33]:
train_tfidf_vectors = {}
for i, index in enumerate(train_index_array):
    vector = train_tfidf_vector_array.getrow(i).toarray()[0]
    train_tfidf_vectors[index] = vector

In [34]:
train_tfidf_vectors[120]

array([0., 0., 0., ..., 0., 0., 0.])

In [35]:
test_index_array, test_vector_array = list(zip(*test_bow_vectors.items()))

# 使用同一個`TfidfTransformer`來轉換testing dataset
# YOUR CODE HERE

transformer = TfidfTransformer()
test_tfidf_vector_array = transformer.fit_transform(test_vector_array)
# END YOUR CODE

In [36]:
test_tfidf_vectors = {}
for i, index in enumerate(test_index_array):
    vector = test_tfidf_vector_array.getrow(i).toarray()
    test_tfidf_vectors[index] = vector

# TFIDF + Group mean vector: 測試

In [37]:
group_vectors = {news_class: [] for news_class in all_news_class}
for index, vector in sorted(train_tfidf_vectors.items()):
    news_class = train_classes[index]
    group_vectors[news_class].append(vector)

group_mean_vector = {}
for news_class, vectors in group_vectors.items():
    group_mean_vector[news_class] = np.mean(vectors, axis=0)
group_mean_vector

{'旅遊': array([0., 0., 0., ..., 0., 0., 0.]),
 '科技': array([0., 0., 0., ..., 0., 0., 0.]),
 '財經': array([0., 0., 0., ..., 0., 0., 0.]),
 '農業': array([0., 0., 0., ..., 0., 0., 0.]),
 '遊戲': array([0.        , 0.        , 0.        , ..., 0.00171764, 0.00152024,
        0.00152024]),
 '體育': array([0.01151249, 0.00394476, 0.00120532, ..., 0.        , 0.        ,
        0.        ])}

In [38]:
def cosine_similarity(bow1, bow2):
    len_bow1 = np.sqrt(np.sum(np.square(bow1)))
    len_bow2 = np.sqrt(np.sum(np.square(bow2)))
    return np.sum(bow1 * bow2) / (len_bow1 * len_bow2)

In [39]:
classification = {news_class: [] for news_class in all_news_class}
for index, vector in sorted(test_tfidf_vectors.items()):
    if np.sum(np.square(vector)) == 0:
        continue

    max_val = -2.0
    max_class = None
    for news_class, ref_vector in group_mean_vector.items():
        val = cosine_similarity(ref_vector, vector)
        if val > max_val:
            max_class = news_class
            max_val = val

    classification[max_class].append(index)

In [40]:
from collections import Counter

for group, ids in classification.items():
    counter = Counter([test_classes[id] for id in ids])
    print('predict', group, ': ', counter)

predict 體育 :  Counter({'體育': 83, '遊戲': 7, '財經': 6, '旅遊': 6, '科技': 3, '農業': 3})
predict 財經 :  Counter({'財經': 76, '科技': 14, '旅遊': 6, '農業': 6, '體育': 4, '遊戲': 2})
predict 科技 :  Counter({'科技': 66, '財經': 11, '農業': 6, '體育': 3, '遊戲': 3, '旅遊': 1})
predict 旅遊 :  Counter({'旅遊': 78, '農業': 10, '科技': 5, '財經': 3, '遊戲': 1})
predict 農業 :  Counter({'農業': 74, '旅遊': 7, '科技': 6, '體育': 4, '財經': 2, '遊戲': 1})
predict 遊戲 :  Counter({'遊戲': 86, '科技': 5, '體育': 4, '財經': 2})


# Visualization

In [41]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2, svd_solver='full')

anchor_classes, anchor_vectors = list(zip(*group_mean_vector.items()))
# 為了看清楚Group Mean Vector怎麼幫助我們做分類問題，我們針對Group Mean Vector (anchor) 來進行PCA降維
# YOUR CODE HERE
reduced_test_tfidf_vector_array = pca.fit(anchor_vectors)
# END YOUR CODE

In [42]:
reduced_test_tfidf_vector_array

PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
    svd_solver='full', tol=0.0, whiten=False)

array([[-0.0237776 , -0.03045768],

       [ 0.03089514, -0.08325018],

       [-0.02799609, -0.04679851],

       ...,

       [ 0.0402831 , -0.04660735],

       [ 0.09522959, -0.04196424],

       [ 0.03325491, -0.03344588]])

In [43]:
import matplotlib
import matplotlib.pyplot as plt

test_class_array = np.vectorize(test_classes.get)(test_index_array)

label_mapping = {
    '體育': 'sport',
    '財經': 'financial',
    '科技': 'tech',
    '旅遊': 'travel',
    '農業': 'agriculture',
    '遊戲': 'game'
}

plt.figure(figsize=(8, 8))
for real_class in all_news_class:
    plt.scatter(reduced_test_tfidf_vector_array[test_class_array == real_class, 0],
                reduced_test_tfidf_vector_array[test_class_array == real_class, 1],
                label=label_mapping[real_class],
                alpha=0.3)
    i = anchor_classes.index(real_class)
    plt.plot([0, reduced_anchor_vectors[i, 0]], [0, reduced_anchor_vectors[i, 1]])
plt.legend(loc="best", shadow=False, scatterpoints=1)

plt.show()

TypeError: ignored

<Figure size 576x576 with 0 Axes>